In [ ]:
# 🌱 HYDROPONICS OPTIMAL CONDITION CHECKER (Final Improved Jupyter Version)
# -------------------------------------------------------------------------

# ✅ Step 1: Import Libraries
import pandas as pd
import numpy as np
import pickle
import os
from difflib import get_close_matches
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# ✅ Step 2: Load Dataset
DATA_PATH = "hydroponics.csv"  # make sure file is in same folder
df = pd.read_csv(DATA_PATH)

print("✅ File Loaded Successfully!")
print("Columns in CSV:", df.columns.tolist())
display(df.head())

# ✅ Step 3: Encode Plant Names
le = LabelEncoder()
df["plant_encoded"] = le.fit_transform(df["plant_name"])

# ✅ Step 4: Prepare Data for Model Training
X = df[["plant_encoded"]]
y = df[["optimal_temp_c", "optimal_ph", "optimal_humidity", "optimal_nutrient_ppm"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Step 5: Train Model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

print("✅ Model trained successfully!")

# ✅ Step 6: Save Model and Label Encoder
SAVE_DIR = "models"
os.makedirs(SAVE_DIR, exist_ok=True)

MODEL_PATH = os.path.join(SAVE_DIR, "hydro_model.pkl")
ENCODER_PATH = os.path.join(SAVE_DIR, "label_encoder.pkl")

with open(MODEL_PATH, "wb") as f:
    pickle.dump(model, f)

with open(ENCODER_PATH, "wb") as f:
    pickle.dump(le, f)

print(f"✅ Model saved to: {MODEL_PATH}")
print(f"✅ Label encoder saved to: {ENCODER_PATH}")

# ✅ Step 7: Load Model (to simulate real-time prediction)
with open(MODEL_PATH, "rb") as f:
    model = pickle.load(f)
with open(ENCODER_PATH, "rb") as f:
    le = pickle.load(f)

print("\n🔁 Model reloaded successfully!")

# ✅ Step 8: Helper Functions

def get_optimal_conditions(plant_name):
    """Return optimal conditions for the selected plant, with fuzzy matching."""
    plant_name_lower = plant_name.lower()
    matches = df[df["plant_name"].str.lower() == plant_name_lower]

    if matches.empty:
        # Try fuzzy matching
        possible = get_close_matches(plant_name_lower, df["plant_name"].str.lower().tolist(), n=1, cutoff=0.6)
        if possible:
            suggestion = possible[0]
            print(f"⚠️ '{plant_name}' not found. Did you mean '{suggestion}'?")
            matches = df[df["plant_name"].str.lower() == suggestion]
        else:
            print(f"❌ Plant '{plant_name}' not found in dataset.")
            return None
    return matches.iloc[0]


def predict_optimal_conditions(plant_name):
    """Use model to predict optimal conditions for the given plant."""
    try:
        plant_label = le.transform([plant_name])[0]
    except ValueError:
        print(f"❌ Unknown plant: '{plant_name}'. Try a valid name.")
        return None

    prediction = model.predict([[plant_label]])[0]
    return {
        "optimal_temp_c": round(prediction[0], 2),
        "optimal_ph": round(prediction[1], 2),
        "optimal_humidity": round(prediction[2], 2),
        "optimal_nutrient_ppm": round(prediction[3], 2)
    }


def suggest_adjustments(user_input, optimal):
    """Suggest actions based on user input vs. optimal."""
    suggestions = []

    # Temperature
    if user_input["temperature_c"] < optimal["optimal_temp_c"]:
        suggestions.append(f"Increase temperature by {round(optimal['optimal_temp_c'] - user_input['temperature_c'], 2)}°C.")
    elif user_input["temperature_c"] > optimal["optimal_temp_c"]:
        suggestions.append(f"Decrease temperature by {round(user_input['temperature_c'] - optimal['optimal_temp_c'], 2)}°C.")

    # pH
    if user_input["ph"] < optimal["optimal_ph"]:
        suggestions.append(f"Raise pH by {round(optimal['optimal_ph'] - user_input['ph'], 2)}.")
    elif user_input["ph"] > optimal["optimal_ph"]:
        suggestions.append(f"Lower pH by {round(user_input['ph'] - optimal['optimal_ph'], 2)}.")

    # Humidity
    if user_input["humidity_pct"] < optimal["optimal_humidity"]:
        suggestions.append(f"Increase humidity by {round(optimal['optimal_humidity'] - user_input['humidity_pct'], 2)}%.")
    elif user_input["humidity_pct"] > optimal["optimal_humidity"]:
        suggestions.append(f"Decrease humidity by {round(user_input['humidity_pct'] - optimal['optimal_humidity'], 2)}%.")

    # Nutrients
    if user_input["nutrient_ppm"] < optimal["optimal_nutrient_ppm"]:
        suggestions.append(f"Add nutrients: increase by {round(optimal['optimal_nutrient_ppm'] - user_input['nutrient_ppm'], 2)} ppm.")
    elif user_input["nutrient_ppm"] > optimal["optimal_nutrient_ppm"]:
        suggestions.append(f"Dilute solution: reduce nutrients by {round(user_input['nutrient_ppm'] - optimal['optimal_nutrient_ppm'], 2)} ppm.")

    if not suggestions:
        suggestions.append("All parameters are within optimal range! 🌿")
    return suggestions


def display_optimal_conditions(optimal):
    """Nicely formatted output for optimal conditions."""
    print("\n🌿 Optimal Conditions:")
    print(f"   🌡️ Temperature: {optimal['optimal_temp_c']}°C")
    print(f"   ⚗️ pH: {optimal['optimal_ph']}")
    print(f"   💧 Humidity: {optimal['optimal_humidity']}%")
    print(f"   🥗 Nutrient Level: {optimal['optimal_nutrient_ppm']} ppm")

# ✅ Step 9: Interactive Test Section
print("\n🌱 HYDROPONICS OPTIMAL CONDITION CHECKER 🌱")
plant_name = input("Enter plant name: ").strip()

try:
    user_input = {
        "plant_name": plant_name,
        "temperature_c": float(input("Enter current temperature (°C): ")),
        "ph": float(input("Enter current pH: ")),
        "humidity_pct": float(input("Enter current humidity (%): ")),
        "nutrient_ppm": float(input("Enter current nutrient level (ppm): "))
    }
except ValueError:
    print("⚠️ Invalid numeric input. Please enter numbers only.")
    raise SystemExit

# ✅ Predict and Suggest
optimal = predict_optimal_conditions(plant_name)

if optimal:
    display_optimal_conditions(optimal)
    suggestions = suggest_adjustments(user_input, optimal)

    print("\n💡 Suggested Adjustments:")
    for s in suggestions:
        print("-", s)


✅ File Loaded Successfully!
Columns in CSV: ['plant_name', 'optimal_temp_c', 'optimal_ph', 'optimal_humidity', 'optimal_nutrient_ppm']


,plant_name,optimal_temp_c,optimal_ph,optimal_humidity,optimal_nutrient_ppm
0,Tomato,22.5,5.55,69.3,331.0
1,Lettuce,29.4,6.45,52.5,618.8
2,Spinach,26.8,5.97,54.8,624.4
3,Basil,25.2,6.26,77.0,682.5
4,Cucumber,19.9,6.86,68.2,735.7


✅ Model trained successfully!
✅ Model saved to: models\hydro_model.pkl
✅ Label encoder saved to: models\label_encoder.pkl

🔁 Model reloaded successfully!

🌱 HYDROPONICS OPTIMAL CONDITION CHECKER 🌱
